In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load the labeled datasets
df_labeled_1 = pd.read_csv('../data_new/supervised-dropped/AB3 - Main 3L.csv')
df_labeled_2 = pd.read_csv('../data_new/supervised-dropped/BS1 - Main 1L.csv')
df_labeled_3 = pd.read_csv('../data_new/supervised-dropped/BS4 - Main 4R.csv')
df_labeled_4 = pd.read_csv('../data_new/supervised-dropped/Sub - Feeder F02.csv')

# Train a Random Forest model on the first labeled dataset
df = df_labeled_1.copy()
df['time'] = pd.to_datetime(df['time'])
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['weekday'] = df['time'].dt.weekday
df = df.drop('time', axis=1)

df['Measure'] = LabelEncoder().fit_transform(df['Measure'])
scaler = StandardScaler()
df['kWh'] = scaler.fit_transform(df[['kWh']])
df['label'] = LabelEncoder().fit_transform(df['label'])

X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Train a Random Forest model on the second labeled dataset
df = df_labeled_2.copy()
df['time'] = pd.to_datetime(df['time'])
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['weekday'] = df['time'].dt.weekday
df = df.drop('time', axis=1)

df['Measure'] = LabelEncoder().fit_transform(df['Measure'])
scaler = StandardScaler()
df['kWh'] = scaler.fit_transform(df[['kWh']])
df['label'] = LabelEncoder().fit_transform(df['label'])

X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.2, random_state=42)

model2 = RandomForestClassifier(n_estimators=100, random_state=42)
model2.fit(X_train, y_train)

# Train a Random Forest model on the third labeled dataset
df = df_labeled_3.copy()
df['time'] = pd.to_datetime(df['time'])
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['weekday'] = df['time'].dt.weekday
df = df.drop('time', axis=1)

df['Measure'] = LabelEncoder().fit_transform(df['Measure'])
scaler = StandardScaler()
df['kWh'] = scaler.fit_transform(df[['kWh']])
df['label'] = LabelEncoder().fit_transform(df['label'])

X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.2, random_state=42)

model3 = RandomForestClassifier(n_estimators=100, random_state=42)
model3.fit(X_train, y_train)

# Save the models
with open('../models/semi_supervised_rfc_model1.pickle', 'wb') as f:
    pickle.dump(model, f)

In [17]:
# Load the new dataframe to be predicted
new_df = pd.read_csv('../data_new/supervised-dropped/BS1 - Main 1L.csv') #  pd.read_csv('../data_new/holdout_v3.csv')

# Preprocess the new data
new_df['time'] = pd.to_datetime(new_df['time'])
new_df['year'] = new_df['time'].dt.year
new_df['month'] = new_df['time'].dt.month
new_df['day'] = new_df['time'].dt.day
new_df['hour'] = new_df['time'].dt.hour
new_df['weekday'] = new_df['time'].dt.weekday
new_df = new_df.drop('time', axis=1)

new_df['Measure'] = LabelEncoder().fit_transform(new_df['Measure'])
scaler = StandardScaler()
new_df['kWh'] = scaler.fit_transform(new_df[['kWh']])

# Predict the labels for the new data using the trained model
new_preds = model.predict(new_df.drop('label', axis=1))

# Print the predicted labels
print(new_preds)
# print(label_encoder.inverse_transform(new_df['label']))


[0 0 0 ... 0 0 0]


In [18]:
from sklearn.metrics import accuracy_score

le = LabelEncoder()
y_encoded = le.fit_transform(new_df['label'])

# Calculate the accuracy score of the model on the new data
accuracy = accuracy_score(y_encoded, new_preds)
print('Accuracy:', accuracy)

Accuracy: 0.9236479321314952


In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import LabelPropagation
import pickle

# Load all data
df_labeled_1 = pd.read_csv('../data_new/supervised-dropped/AB3 - Main 3L.csv')
df_labeled_2 = pd.read_csv('../data_new/supervised-dropped/BS1 - Main 1L.csv')
df_labeled_3 = pd.read_csv('../data_new/supervised-dropped/BS4 - Main 4R.csv')
df_labeled_4 = pd.read_csv('../data_new/supervised-dropped/Sub - Feeder F02.csv')
df_unlabeled = pd.read_csv('../data_new/holdout_v3.csv')

# Concatenate labeled and unlabeled data into a single DataFrame
df = pd.concat([df_labeled_1, df_labeled_2, df_labeled_3, df_labeled_4, df_unlabeled], ignore_index=True)

# Split the data into labeled and unlabeled sets
labeled_mask = df['label'].notnull()
df_labeled = df[labeled_mask].copy()
df_unlabeled = df[~labeled_mask].copy()

# Preprocess the data
df_labeled['time'] = pd.to_datetime(df_labeled['time'])
df_labeled['year'] = df_labeled['time'].dt.year
df_labeled['month'] = df_labeled['time'].dt.month
df_labeled['day'] = df_labeled['time'].dt.day
df_labeled['hour'] = df_labeled['time'].dt.hour
df_labeled['weekday'] = df_labeled['time'].dt.weekday
df_labeled = df_labeled.drop('time', axis=1)

df_labeled['Measure'] = LabelEncoder().fit_transform(df_labeled['Measure'])
scaler = StandardScaler()
df_labeled['kWh'] = scaler.fit_transform(df_labeled[['kWh']])
df_labeled['label'] = LabelEncoder().fit_transform(df_labeled['label'])

df_unlabeled['time'] = pd.to_datetime(df_unlabeled['time'])
df_unlabeled['year'] = df_unlabeled['time'].dt.year
df_unlabeled['month'] = df_unlabeled['time'].dt.month
df_unlabeled['day'] = df_unlabeled['time'].dt.day
df_unlabeled['hour'] = df_unlabeled['time'].dt.hour
df_unlabeled['weekday'] = df_unlabeled['time'].dt.weekday
df_unlabeled = df_unlabeled.drop('time', axis=1)

df_unlabeled['Measure'] = LabelEncoder().fit_transform(df_unlabeled['Measure'])
df_unlabeled['kWh'] = scaler.transform(df_unlabeled[['kWh']])

# Train a semi-supervised learning model
model = LabelPropagation(kernel='knn', n_neighbors=7)
X_labeled = df_labeled.drop('label', axis=1)
y_labeled = df


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.